# Melbourne Pedestrian Data Collection
###### By Cooper Denny

Last updated: 

- Thurs 17 October 2024


In [3]:
import pandas as pd
import numpy as np
import requests
from datetime import datetime

## Introduction
This report aims to gather and tranform pedestrian data for the city of Melbourne from January 2010 to September 2024. The dataset is obtained from the Melbourne Pedestrian Counting System, which records pedestrian counts across various sensors in the city. The data is available in CSV format for each year and month. The URLs for the data files are constructed dynamically based on the year and month.

In [4]:
# Define the months for use in file names
months = ["January", "February", "March", "April", "May", "June",
          "July", "August", "September", "October", "November", "December"]

# Initialize an empty dataframe to store pedestrian data
pedestrian_data = pd.DataFrame()

# Helper function to download and clean data
def download_and_clean_data(url):
    try:
        data_year_month = pd.read_csv(url)
        data_year_month[data_year_month.columns[1:]] = data_year_month.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')
        data_year_month['Date'] = pd.to_datetime(data_year_month['Date'], dayfirst=True, errors='coerce')
        return data_year_month
    except Exception as e:
        print(f"Error downloading data from {url}: {e}")
        return pd.DataFrame()  # Return an empty DataFrame in case of error

## Data Aggregation Process
The following code loops through each year and month, downloading the respective pedestrian data files, and appends them to a single DataFrame.

In [5]:
# Loop through years 2010 to 2012
all_data = []
for y in range(2010, 2013):
    url = f"https://www.pedestrian.melbourne.vic.gov.au/datadownload/{y}.csv"
    data_year = download_and_clean_data(url)
    all_data.append(data_year)

# Loop through years 2013 to 2023
for y in range(2013, 2024):
    for m in range(1, 13):
        url = f"https://www.pedestrian.melbourne.vic.gov.au/datadownload/{months[m-1]}_{y}.csv"
        data_year_month = download_and_clean_data(url)
        all_data.append(data_year_month)
        
# Loop through year 2024
for y in range(2013, 2025):
    for m in range(1, 10):
        url = f"https://www.pedestrian.melbourne.vic.gov.au/datadownload/{months[m-1]}_{y}.csv"
        data_year_month = download_and_clean_data(url)
        all_data.append(data_year_month)
    
pedestrian_data = pd.concat(all_data, ignore_index=True)

## Missing Data Check
We check for any missing data in the 'Date' column. If any missing dates are found, a warning will be printed. Otherwise, the analysis continues without interruption.

In [6]:
# Check for missing dates in the original dataframe
missing_dates_main = pedestrian_data['Date'].isna().sum()

if missing_dates_main > 0:
    print(f"Warning: {missing_dates_main} missing dates found in the original dataframe.")
else:
    # Create a dataframe with the same columns as the last dataset
    pedestrian_data_columns = pedestrian_data.loc[:, data_year_month.columns]

    # Check for missing dates in the subset dataframe
    missing_dates_subset = pedestrian_data_columns['Date'].isna().sum()

    if missing_dates_subset > 0:
        print(f"Warning: {missing_dates_subset} missing dates found in the subset dataframe.")
    else:
        pass  # No missing dates, just continue

## Data Preparation for Analysis
In this section, we combine the 'Date' and 'Hour' columns to create a 'Datetime' column for better time-series analysis. The data is also converted into a long format, which allows us to analyze pedestrian counts based on the sensor locations.

In [7]:
# Combine 'Date' and 'Hour' columns into a 'Datetime' column efficiently
pedestrian_data_columns['Hour'] = pedestrian_data_columns['Hour'].apply(lambda x: f"{int(x):02d}:00")

# Use pd.concat to avoid fragmentation by joining all columns at once
pedestrian_data_combined = pd.concat([
    pedestrian_data_columns[['Date', 'Hour']],
    pedestrian_data_columns.iloc[:, 2:-1]  # Selecting all other columns except 'Date' and 'Hour'
], axis=1)

# Create 'Datetime' from 'Date' and 'Hour'
pedestrian_data_combined['Datetime'] = pd.to_datetime(pedestrian_data_combined['Date'].astype(str) + ' ' + pedestrian_data_combined['Hour'])

# Select required columns (Datetime + other relevant columns)
pedestrian_data_combined = pedestrian_data_combined[['Datetime'] + list(pedestrian_data_combined.columns[2:-1])]

# Convert data to long format
pedestrian_data_long = pedestrian_data_combined.melt(id_vars=['Datetime'], var_name='Sensor', value_name='Hourly Pedestrian Count')

# Display the transformed data
pedestrian_data_long.head()

,Datetime,Sensor,Hourly Pedestrian Count
0,2010-01-01 00:00:00,Bourke Street Mall (North),465.0
1,2010-01-01 01:00:00,Bourke Street Mall (North),262.0
2,2010-01-01 02:00:00,Bourke Street Mall (North),273.0
3,2010-01-01 03:00:00,Bourke Street Mall (North),256.0
4,2010-01-01 04:00:00,Bourke Street Mall (North),209.0


## Merging Sensor Locations with Pedestrian Data

To enhance the pedestrian dataset with spatial information, we needed to map each sensor to its respective geographical location. Using Google Maps, I was able to pinpoint the approximate latitude and longitude coordinates for each sensor location. 

In [8]:
# Extract unique sensor names
sensors = pedestrian_data_long['Sensor'].unique()

# Latitude and Longitude vectors for each sensor location
latitude = [
    -37.81358, -37.81375, -37.81111, -37.81489, -37.81873, -37.81912, -37.81816, -37.82279, -37.81986, -37.81877, 
    -37.81558, -37.81451, -37.81233, -37.82008, -37.81069, -37.81360, -37.81349, -37.81238, -37.81169, -37.81249, 
    -37.81789, -37.81903, -37.81885, -37.82412, -37.80879, -37.80604, -37.82136, -37.81970, -37.81113, -37.80170, 
    -37.81574, -37.81480, -37.81551, -37.82019, -37.81647, -37.80303, -37.81726, -37.81371, -37.80999, -37.80016, 
    -37.79847, -37.79689, -37.81416, -37.81662, -37.80234, -37.81156, -37.80629, -37.80723, -37.79808, -37.80840, 
    -37.81251, -37.81565, -37.80390, -37.81235, -37.81773, -37.81689, -37.80809, -37.81659, -37.80760, -37.80997, 
    -37.80989, -37.81330, -37.79987, -37.81552, -37.81056, -37.81694, -37.81678, -37.81686, -37.80444, -37.81229, 
    -37.81724, -37.81690, -37.81517, -37.79451, -37.81437, -37.81473, -37.81795, -37.81803, -37.79444, -37.80306, 
    -37.80447, -37.81248, -37.81296, -37.81194, -37.81632, -37.81749, -37.81997, -37.81969, -37.81900, -37.81964, 
    -37.81841, -37.82631, -37.82046, -37.80676, -37.80667, -37.80658, -37.80652, -37.82183, -37.80705
]

longitude = [
    144.96515, 144.96507, 144.96422, 144.96611, 144.96782, 144.96551, 144.97155, 144.94714, 144.95100, 144.94706, 
    144.94014, 144.94294, 144.95655, 144.96297, 144.96446, 144.97327, 144.97299, 144.96548, 144.96819, 144.96781, 
    144.96502, 144.95460, 144.95451, 144.95607, 144.96048, 144.95634, 144.96885, 144.96864, 144.96655, 144.96663, 
    144.96677, 144.97452, 144.97361, 144.96508, 144.96121, 144.96666, 144.96721, 144.96991, 144.97227, 144.96387, 
    144.96413, 144.96441, 144.96598, 144.96692, 144.96154, 144.96207, 144.95914, 144.95959, 144.96725, 144.95908, 
    144.96181, 144.96551, 144.96313, 144.96155, 144.95030, 144.95349, 144.96301, 144.96689, 144.96305, 144.96137, 
    144.96229, 144.96679, 144.95790, 144.96661, 144.96442, 144.96549, 144.96580, 144.96566, 144.94937, 144.97136, 
    144.96864, 144.95431, 144.97460, 144.93032, 144.94431, 144.94463, 144.96616, 144.96505, 144.92953, 144.94907, 
    144.94926, 144.95697, 144.95681, 144.95615, 144.97087, 144.97334, 144.95829, 144.96788, 144.94619, 144.96332, 
    144.97349, 144.96156, 144.94128, 144.95998, 144.95999, 144.96003, 144.96004, 144.95543, 144.96306
]

# Creating DataFrame for sensor locations
sensor_locations = pd.DataFrame({
    'Sensor': sensors,
    'Latitude': latitude,
    'Longitude': longitude
})

# Display the sensor locations data
sensor_locations.head()

,Sensor,Latitude,Longitude
0,Bourke Street Mall (North),-37.81358,144.96515
1,Bourke Street Mall (South),-37.81375,144.96507
2,Melbourne Central,-37.81111,144.96422
3,Town Hall (West),-37.81489,144.96611
4,Princes Bridge,-37.81873,144.96782


Next, we merged this new DataFrame with the original pedestrian data, linking each sensor with its corresponding location.

In [9]:
# Merge the pedestrian data with sensor locations
pedestrian_data_merged = pedestrian_data_long.merge(sensor_locations, on='Sensor', how='inner')

# Display the first few rows of the pedestrian data
pedestrian_data_merged.head()

,Datetime,Sensor,Hourly Pedestrian Count,Latitude,Longitude
0,2010-01-01 00:00:00,Bourke Street Mall (North),465.0,-37.81358,144.96515
1,2010-01-01 01:00:00,Bourke Street Mall (North),262.0,-37.81358,144.96515
2,2010-01-01 02:00:00,Bourke Street Mall (North),273.0,-37.81358,144.96515
3,2010-01-01 03:00:00,Bourke Street Mall (North),256.0,-37.81358,144.96515
4,2010-01-01 04:00:00,Bourke Street Mall (North),209.0,-37.81358,144.96515


The final step involved exporting the cleaned and merged dataset as a CSV file named pedestrian_data_cleaned.csv. This file contains pedestrian counts along with sensor locations, organized by date and time, and is ready for further analysis or visualization.

In [10]:
#pedestrian_data_merged.to_csv('pedestrian_data_cleaned.csv', index=False)

The cleaned dataset, now enhanced with spatial and temporal information, is ready for advanced analyses such trend detection, spatial analysis, or visualization tasks. 

## Conclusion
With the dynamic URL construction and spatial merging of sensor data, we were able to analyze hourly counts for different locations over the years. Further analysis could include forecasting pedestrian movement and identifying peak hours and areas of high foot traffic. Future steps might also involve visualizing this data on an interactive map or using machine learning models to predict pedestrian trends.